# Prepare forcing raw data CAMELS-GB
## CEH-GEAR: pr, CHESS-PE: pet, CHESS-met: tas

In [1]:
# This cell is only used to suppress some distracting output messages
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import xarray as xr
from bng_latlon import OSGB36toWGS84

from glob import glob
from pathlib import Path

# Set Paths

In [3]:
# Snellius paths
ROOT = '/gpfs/work1/0/wtrcycle/users/jaerts/camels_uk/'
FORCING_RAW = f'{ROOT}/forcing_raw/'
FORCING = f'{ROOT}/forcing/'

# Prepare tas

In [4]:
# Read forcing files
forcing_files = glob(f'{FORCING_RAW}/tas/*')
ds = xr.open_mfdataset(forcing_files)

# Assign lat lon instead of native GB coords
ds = ds.assign_coords({'x':ds.lon[0,:].values, 'y':ds.lat[:,0].values})

# Drop redundant variables
ds = ds.drop_vars(['lat','lon','time_bnds', 'x_bnds', 'y_bnds', 'crsOSGB']) 

# Rename to lat lon
ds = ds.rename({'x':'lon','y':'lat'})

# Store output
ds.to_netcdf(f'{FORCING}/chess-met_tas_gb_1km_daily_2000_2017.nc')

# Prepare pet

In [5]:
# Read forcing files
forcing_files = glob(f'{FORCING_RAW}/pet/*')
ds = xr.open_mfdataset(forcing_files)

# Assign lat lon instead of native GB coords
ds = ds.assign_coords({'x':ds.lon[0,:].values, 'y':ds.lat[:,0].values})

# Drop redundant variables
ds = ds.drop_vars(['lat','lon','time_bnds', 'x_bnds', 'y_bnds', 'crsOSGB']) 

# Rename to lat lon
ds = ds.rename({'x':'lon','y':'lat'})

# Store output
ds.to_netcdf(f'{FORCING}/chess-pe_pet_gb_1km_daily_2000_2017.nc')

# Prepare pr

In [23]:
# Read forcing files
forcing_files = glob(f'{FORCING_RAW}/pr/*')
ds = xr.open_mfdataset(forcing_files)

In [24]:
# Convert to lonlat from OBSGB36

lons=[]
lats=[]

for i in ds.x.values:
    j=0
    lons.append(OSGB36toWGS84(i,j)[1])
        
for j in ds.y.values:
    i=0
    lats.append(OSGB36toWGS84(i,j)[0])        
        
ds = ds.assign_coords({
        "x": lons,
        "y": lats,
        })   

In [25]:
# Drop redundant variables
ds = ds.drop_vars(['lat','lon','min_dist','crs'])

# Rename to lat lon
ds = ds.rename({'x':'lon','y':'lat'})

# Rechunk dataset
ds = ds.chunk(chunks='auto')
ds = ds.unify_chunks()

# Store output
ds.to_netcdf(f'{FORCING}/ceh-gear_pr_gb_1km_daily_2000_2019.nc')